# Dynamic Tariff Model & ML Comparison

This notebook implements a simulation of dynamic electricity tariffs based on real-world Day-Ahead market data from Awattar (Germany). It compares different tariff models against a static baseline and finally evaluates the predictability of the best tariff using Machine Learning models.

## Objectives
1. **Data Acquisition**: Fetch market prices from Awattar API.
2. **Load Profiling**: Generate a synthetic 24h household load profile (~10 kWh/day).
3. **Tariff Modeling**:
    - **Baseline**: Static (0.35 €/kWh)
    - **Pass-through**: Market Price + Margin (0.03 €/kWh)
    - **Capped**: Pass-through with Cap (0.40 €/kWh)
    - **Smoothed**: Rolling average of market price (3h) + Margin
4. **Evaluation**: Compare Customer Cost, Volatility, and Vendor Profit (Simple Proxy).
5. **ML Comparison**: Benchmark "Best Model" prices against ML predictions (Random Forest, KNN).

In [ ]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from IPython.display import Markdown, display

# Settings
sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

print("Libraries imported successfully.")

## 1. Data Acquisition (Awattar API)
We fetch the latest available market data (Day-Ahead) from the Awattar API for Germany.

In [ ]:
def fetch_awattar_data():
    # Fetching historical data
    # Note: Using 6 years to cover full history if needed, as requested by user activity
    end_ts = int(pd.Timestamp.now().timestamp() * 1000)
    # Going back 4 years guarantees full coverage of 2022. 
    start_ts = int((pd.Timestamp.now() - pd.DateOffset(years=6)).timestamp() * 1000)
    
    all_data = []
    current = start_ts
    # chunk_size_ms = 60 * 24 * 3600 * 1000 # 60 days chunks to reduce request count
    chunk_size_ms = 100 * 24 * 3600 * 1000 # 100 days chunks based on successful test
    
    print(f"Fetching data from {pd.Timestamp(start_ts, unit='ms')} to {pd.Timestamp(end_ts, unit='ms')}...")
    
    while current < end_ts:
        next_step = min(current + chunk_size_ms, end_ts)
        url = f"https://api.awattar.de/v1/marketdata?start={current}&end={next_step}"
        
        # Retry logic for rate limits
        max_retries = 3
        for attempt in range(max_retries):
            try:
                response = requests.get(url)
                if response.status_code == 200:
                    data = response.json().get('data', [])
                    all_data.extend(data)
                    break # Success, move to next chunk
                elif response.status_code == 429:
                    print(f"Rate limited (429) on chunk {current}. Waiting 5s...")
                    time.sleep(5) # Backoff significantly
                else:
                    print(f"Error fetching chunk {current}: Status {response.status_code}")
                    break # Non-retriable error
            except Exception as e:
                print(f"Exception fetching chunk {current}: {e}")
                time.sleep(1)
        
        current = next_step
        # Standard polite sleep between successful requests
        time.sleep(1.0) 
        
    if not all_data:
        raise ValueError("No data returned from Awattar API")
        
    df = pd.DataFrame(all_data)
    # Convert timestamp (ms) to datetime
    df['start_time'] = pd.to_datetime(df['start_timestamp'], unit='ms')
    df['end_time'] = pd.to_datetime(df['end_timestamp'], unit='ms')
    
    # Market price is usually in Eur/MWh, convert to Eur/kWh
    # 1 Eur/MWh = 0.001 Eur/kWh
    df['market_price_eur_kwh'] = df['marketprice'] / 1000.0
    
    df = df.set_index('start_time').sort_index()
    # Remove duplicates just in case
    df = df[~df.index.duplicated(keep='first')]
    return df[['market_price_eur_kwh']]

df_market = fetch_awattar_data()
print(f"Fetched {len(df_market)} rows of market data.")
df_market.head()

## 2. Generate Synthetic Load Profile
We simulate a typical household load (~10 kWh/day) with morning and evening peaks.

In [ ]:
def generate_synthetic_load(index, total_daily_kwh=10):
    # Create a base pattern for 24 hours
    hour_of_day = index.hour
    
    # Simple heuristic for household: Low at night, hump in morning, dip in afternoon, peak in evening
    # Weights for each hour 0-23
    weights = np.array([
        0.2, 0.2, 0.2, 0.2, 0.3, 0.5, # 00-05: Night
        1.0, 1.2, 1.0, 0.8, 0.7, 0.6, # 06-11: Morning Peak & decrease
        0.6, 0.6, 0.7, 0.9, 1.5, 2.0, # 12-17: Afternoon & ramping up
        2.2, 2.0, 1.8, 1.2, 0.6, 0.3  # 18-23: Evening Peak & wind down
    ])
    
    # Map weights to the dataframe index
    load_weights = np.array([weights[h] for h in hour_of_day])
    
    # Normalize to sum to 1.0 then multiply by total kWh required * number of days approx
    # Actually we want the sum over one day to be 10 kWh.
    # Since we might have multiple days in 'index', we handle normalization carefully.
    
    # Calculate load per hour
    # Average sum of weights per day is sum(weights) = 21.3
    # Scaling factor = 10 kWh / 21.3
    scaling_factor = total_daily_kwh / weights.sum()
    
    simulated_load = load_weights * scaling_factor
    
    # Add some random noise
    noise = np.random.normal(0, 0.05, size=len(simulated_load))
    simulated_load = np.maximum(simulated_load + noise, 0) # Ensure no negative load
    
    return simulated_load

df_market['load_kwh'] = generate_synthetic_load(df_market.index)
print(f"Average Daily Load: {(df_market['load_kwh'].sum() / (len(df_market)/24)):.2f} kWh")

plt.figure(figsize=(10, 4))
plt.plot(df_market.index, df_market['load_kwh'], label='Load (kWh)')
plt.title("Synthetic Household Load Profile (Full History)")
plt.ylabel("kWh")
plt.legend()
plt.show()

# Zoomed-in view for 1 week to show the pattern clearly
subset_week = df_market.iloc[:168] # First 7 days
plt.figure(figsize=(10, 4))
plt.plot(subset_week.index, subset_week['load_kwh'], label='Load (kWh) - 1 Week Zoom', color='orange')
plt.title("Synthetic Household Load Profile (1 Week Zoom)")
plt.ylabel("kWh")
plt.legend()
plt.show()

# Zoomed-in view for 1 Day to show the daily profile details
subset_day = df_market.iloc[:24] # First 24 hours
plt.figure(figsize=(10, 4))
plt.plot(subset_day.index, subset_day['load_kwh'], label='Load (kWh) - 1 Day Zoom', color='green', marker='o')
plt.title("Synthetic Household Load Profile (1 Day Zoom - 24h)")
plt.ylabel("kWh")
plt.legend()
plt.show()

## 3. Define Tariff Models
We define the logic for calculating the final consumer price for each model.
Assumptions:
- **Margin**: 0.03 €/kWh (Pass-through margin).
- **Cap**: 0.40 €/kWh for the Capped model.
- **Static Rate**: 0.35 €/kWh.

In [ ]:
# Parameters
STATIC_RATE = 0.35
MARGIN = 0.03            # User defined margin
CAP_PRICE = 0.40         # User defined cap
SMOOTHING_WINDOW = 3     # Rolling average window (hours)

# 1. Baseline: Static
df_market['price_static'] = STATIC_RATE

# 2. Pass-through: Market + Margin
df_market['price_passthrough'] = df_market['market_price_eur_kwh'] + MARGIN

# 3. Capped: Min(Pass-through, Cap)
df_market['price_capped'] = df_market['price_passthrough'].clip(upper=CAP_PRICE)

# 4. Smoothed: Rolling Mean of Market + Margin
df_market['price_smoothed'] = (
    df_market['market_price_eur_kwh'].rolling(window=SMOOTHING_WINDOW, min_periods=1).mean() 
    + MARGIN
)

# Visualization of Prices
plt.figure(figsize=(12, 6))
plt.plot(df_market.index, df_market['price_static'], label='Static', linestyle='--')
plt.plot(df_market.index, df_market['price_passthrough'], label='Pass-through')
plt.plot(df_market.index, df_market['price_capped'], label='Capped')
plt.plot(df_market.index, df_market['price_smoothed'], label='Smoothed')
plt.title("Tariff Price Comparison")
plt.ylabel("Price (€/kWh)")
plt.legend()
plt.show()

## 4. Simulation & Metrics
Calculate three key metrics:
1. **Customer Cost**: `Sum(Load * TariffPrice)`
2. **Volatility**: Standard Deviation of `TariffPrice`
3. **Vendor Profit (Simple Proxy)**: `(TariffPrice - MarketPrice) * Load`

In [ ]:
results = []
models = ['static', 'passthrough', 'capped', 'smoothed']

for m in models:
    col_name = f'price_{m}'
    
    # Customer Cost
    total_cost = (df_market['load_kwh'] * df_market[col_name]).sum()
    avg_daily_cost = total_cost / (len(df_market) / 24)
    
    # Price Volatility (Std Dev)
    volatility = df_market[col_name].std()
    
    # Vendor Profit (Simple Proxy)
    # Profit = (Price_Charged - Market_Price) * Load
    hourly_profit = (df_market[col_name] - df_market['market_price_eur_kwh']) * df_market['load_kwh']
    total_profit = hourly_profit.sum()
    
    results.append({
        'Model': m.capitalize(),
        'Total Cost (€)': round(total_cost, 2),
        'Volatility (Std)': round(volatility, 4),
        'Vendor Profit (€)': round(total_profit, 2)
    })

df_results = pd.DataFrame(results)
# Calculate Savings % for each model vs Static
static_cost = df_results.loc[df_results['Model'] == 'Static', 'Total Cost (€)'].values[0]
df_results['Savings vs Static (%)'] = round((static_cost - df_results['Total Cost (€)']) / static_cost * 100, 2)

display(df_results)

### **Model Selection**
Based on the results above, we define the "Best Model".

**Objective**: Find the **Best Compromise** (Balanced Business Model).
- Customer: Needs savings > 20% vs Static + **Stability**.
- Vendor: Maximized profit.

We select the model that maximizes the **Stability Efficiency Score**: 
`Efficiency = Vendor Profit / Volatility`

This penalizes models with high volatility (like Pass-through) and rewards those with high profit and smoothness.

In [ ]:
# --- Model Selection Logic (Stability-Adjusted Approach) ---
# Criteria:
# 1. Must provide significant savings to customer (> 20% vs Static).
# 2. Maximize (Profit / Volatility)

print("--- Model Selection (Stability-Adjusted) ---")
target_savings_pct = 20

# Score = Profit / Volatility
# Add small epsilon to volatility to avoid division by zero (for static)
df_results['Stability Score'] = df_results['Vendor Profit (€)'] / (df_results['Volatility (Std)'] + 0.0001)

display(df_results[['Model', 'Vendor Profit (€)', 'Volatility (Std)', 'Stability Score']])

# Filter for models that meet savings target (excluding Static itself)
valid_models = df_results[
    (df_results['Savings vs Static (%)'] > target_savings_pct) & 
    (df_results['Model'] != 'Static')
]

if not valid_models.empty:
    # Select the one with highest Stability Score
    best_model_row = valid_models.sort_values('Stability Score', ascending=False).iloc[0]
    best_model_name = best_model_row['Model']
    reason = "Highest Balance of Profit & Stability (Profit/Volatility)"
else:
    # Fallback: Just pick the one with lowest Customer Cost
    print(f"No model meets the >{target_savings_pct}% savings target with the default params.")
    best_model_row = df_results.sort_values('Total Cost (€)').iloc[0]
    best_model_name = best_model_row['Model']
    reason = "Lowest Customer Cost (Fallback)"

best_model_col = f"price_{best_model_name.lower()}"

print(f"Selected Best Model: {best_model_name}")
print(f"Reason: {reason}")
print(best_model_row)

# --- 4.1 Business Model Optimization (Theoretical Analysis) ---
# We still keep this to show WHAT IF we optimized the margin
print("\n--- Theoretical Optimization (Sweet Spot Analysis) ---")
margins_to_test = np.arange(0.02, 0.15, 0.005)
optimization_results = []

for m_margin in margins_to_test:
    price_scenario = df_market['market_price_eur_kwh'] + m_margin
    scen_cost = (df_market['load_kwh'] * price_scenario).sum()
    scen_savings_pct = (static_cost - scen_cost) / static_cost * 100
    scen_profit = (m_margin * df_market['load_kwh'].sum())
    
    optimization_results.append({
        'Margin (€/kWh)': round(m_margin, 3),
        'Customer Cost (€)': round(scen_cost, 2),
        'Savings vs Static (%)': round(scen_savings_pct, 2),
        'Vendor Profit (€)': round(scen_profit, 2)
    })

df_opt = pd.DataFrame(optimization_results)
sweet_spots = df_opt[df_opt['Savings vs Static (%)'] > 20]

if not sweet_spots.empty:
    best_theoretical = sweet_spots.sort_values('Vendor Profit (€)', ascending=False).iloc[0]
    print(f"Theoretical Sweet Spot found: Margin {best_theoretical['Margin (€/kWh)']} €/kWh")
    print(f"Potential Profit: {best_theoretical['Vendor Profit (€)']} €, Savings: {best_theoretical['Savings vs Static (%)']}%\n")
else:
    print("No theoretical sweet spot found.")

## 5. Machine Learning Comparison
Can we predict the prices of the "Best Model"? If the prices are highly predictable, customers (or batteries) can optimize usage even better.

We will train Random Forest and KNN models to predict upcoming prices based on time-features.

**Features**: Hour of Day, Day of Week, Previous Hour Price (lag).

In [ ]:
# Prepare Dataset for ML
# Target: Best Model Price
# Features: Hour, Lagged Price

df_ml = df_market.copy()
df_ml['hour'] = df_ml.index.hour
df_ml['day_of_week'] = df_ml.index.dayofweek
df_ml['lag_1h'] = df_ml[best_model_col].shift(1)

# Drop NaN created by shift
df_ml = df_ml.dropna()

features = ['hour', 'day_of_week', 'lag_1h']
target = best_model_col

X = df_ml[features]
y = df_ml[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# 1. Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# 2. KNN
knn = KNeighborsRegressor(n_neighbors=5, n_jobs=-1)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

# Evaluation
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
rmse_knn = np.sqrt(mean_squared_error(y_test, y_pred_knn))

print(f"Random Forest RMSE: {rmse_rf:.4f}")
print(f"KNN RMSE: {rmse_knn:.4f}")

# Plotting predictions vs actual for the test set (Subset for visualization)
# Plotting last 7 days only to avoid visual clutter
subset_days = 7 * 24
plt.figure(figsize=(12, 6))
plt.plot(y_test.index[-subset_days:], y_test[-subset_days:], label=f'Actual {best_model_name}', linewidth=2)
plt.plot(y_test.index[-subset_days:], y_pred_rf[-subset_days:], label='Random Forest Pred', linestyle='--')
plt.plot(y_test.index[-subset_days:], y_pred_knn[-subset_days:], label='KNN Pred', linestyle=':')
plt.title(f"ML Price Prediction for {best_model_name} Tariff (Last 7 Days)")
plt.ylabel("Price (€/kWh)")
plt.legend()
plt.show()

In [ ]:
from IPython.display import Markdown, display

# 1. Summary of Best Model
best_cost = df_results.loc[df_results['Model'] == best_model_name, 'Total Cost (€)'].values[0]
static_cost = df_results.loc[df_results['Model'] == 'Static', 'Total Cost (€)'].values[0]
savings_euro = static_cost - best_cost
savings_pct = (savings_euro / static_cost) * 100

summary_text = f"""
## Conclusion (Data-Driven)

### 1. Tariff Simulation Results
*   **Best Model**: **{best_model_name}**
    *   **Reason**: {reason}
    *   This model offers the best compromise: Good profit, high savings, and greater price stability compared to pure pass-through.
*   **Total Cost**: **{best_cost:.2f} €** (vs Static: {static_cost:.2f} €)
*   **Customer Savings**: **{savings_pct:.1f}%** ({savings_euro:.2f} €)

### 2. Business Viability
"""

if 'best_theoretical' in locals():
    summary_text += f"""
*   Theoretical Analysis: Increasing the margin to **{best_theoretical['Margin (€/kWh)']:.3f} €/kWh** would maximize profit while maintaining >20% savings.
"""
else:
    summary_text += "\n*   Optimization analysis found no theoretical sweet spot better than defaults.\n"

summary_text += f"""
### 3. Price Predictability (ML)
*   **Random Forest RMSE**: **{rmse_rf:.4f}**
*   **KNN RMSE**: **{rmse_knn:.4f}**
*   The **{'Random Forest' if rmse_rf < rmse_knn else 'KNN'}** model performed better.
"""

display(Markdown(summary_text))